# Performance Analysis of Low-Flux Least-Squares Single-Pixel Imaging
## by D. Shin, J. H. Shapiro, V. K. Goyal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.misc import imread
% matplotlib inline

def mse_asymptotic(x,n,m,p):
    return (((n-1)*p+1-p)/float(n*p+1-p))*(n*sum(x)/float((1-p)*m))

def mse_proposed(x,n,m,p):
    return (n-1)*sum(x)/((1-float(p))*(m-n))


### comparing MSE estimators 

In [ ]:

# values of (n,p) pairs
np_pair = [(100,0.2), (100,0.5), (100,0.8), (20,0.2), (20,0.5), (20,0.8)]

# values of q
v_q = np.linspace(0.1,0.9,20)


n_monte = 20 # num monte carlo trials (set ~1000 to see good convergence)

print '* * * '
print 'num. monte carlo trials for true MSE = '+str(n_monte)
print '* * * '

save_logmse = []
save_logmse_approx = []
save_logmse_proposed = []
for np_val in np_pair:
    
    n = np_val[0]
    p = np_val[1]    
    print 'signal dimension =  '+str(n)
    print 'modulation prob  =  '+str(p)
    print ''

    v_mse_proposed = []
    v_mse_approx = []
    v_mse = []
    for q in tqdm(v_q):
        m = int(n/q)

        v_monte_mse = []
        v_monte_mse_approx = []
        v_monte_mse_proposed = []
        for i in range(n_monte):
            x = np.random.rand(n)*10
            A = (np.random.rand(m,n)<p).astype(float)
            y = np.random.poisson(np.array(np.matrix(A)*np.matrix(x).T))
            x_est = np.array((np.matrix(np.linalg.pinv(A))*y).T)[0]
            v_monte_mse.append(np.linalg.norm(x_est-x)**2) 
            v_monte_mse_approx.append(mse_asymptotic(x,n,m,p))
            v_monte_mse_proposed.append(mse_proposed(x,n,m,p))
        v_mse.append(np.mean(v_monte_mse))
        v_mse_approx.append(np.mean(v_monte_mse_approx))
        v_mse_proposed.append(np.mean(v_monte_mse_proposed))
    v_logmse = np.log10(v_mse)
    v_logmse_approx = np.log10(v_mse_approx)
    v_logmse_proposed = np.log10(v_mse_proposed)    
    save_logmse.append(v_logmse)
    save_logmse_approx.append(v_logmse_approx)
    save_logmse_proposed.append(v_logmse_proposed)
    
print 'finished simulation'


In [ ]:

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

limx = [0.1,0.9]
limy = [1,5]
fgsz = (3.5,3)

for i in range(len(np_pair)):
    n_val = np_pair[i][0]
    p_val = np_pair[i][1]
    
    print 'signal dimension =  '+str(n_val)
    print 'modulation prob  =  '+str(p_val)
    print ''
    plt.figure(figsize=fgsz)
    plt.hold('true')
    plt.plot(v_q,save_logmse[i],'k--',linewidth=3,ms=5)
    plt.plot(v_q,save_logmse_approx[i],'b-',ms=5)
    plt.plot(v_q,save_logmse_proposed[i],'r-',ms=5)
    plt.xlabel(r'$q$',fontsize=14,labelpad=-1)
    plt.ylabel(r'log-MSE',fontsize=14,labelpad=-1)
    #plt.title(r'$n=$'+str(n_val)+', $p=$'+str(p_val), fontsize=15)
    plt.text(0.15,4.5,r'$n=$'+str(n_val)+', $p=$'+str(p_val),\
             fontsize=15,horizontalalignment='left',fontweight='bold')
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.xlim(limx)
    plt.ylim(limy)
    #plt.savefig('results/results_sim_n'+str(n_val)+'_p'+str(p_val)+'.eps', format="eps",bbox_inches='tight')

    plt.show()
